In [ ]:
import sys
import gymnasium
import os
import pandas as pd
from matplotlib import pyplot as plt

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor

from building_energy_storage_simulation import BuildingSimulation, Environment

# Applying Reiforcement Learning Using Stable Baselines 3


In [ ]:
RL_PATH = 'rl_example/'
os.makedirs(RL_PATH, exist_ok=True)

# Create Environment
sim = BuildingSimulation()
env = Environment(sim)
initial_obs, info = env.reset()
print(initial_obs)

In [ ]:
# Wrap with Monitor() so a log of the training is saved 
env = Monitor(env, filename=RL_PATH)
# Warp with DummyVecEnc() so the observations and reward can be normalized using VecNormalize()
env = DummyVecEnv([lambda: env])
env = VecNormalize(env, norm_obs=True, norm_reward=True)

In [ ]:
# Train with PPO :-)
model = PPO("MlpPolicy", env, verbose=1, gamma=0.95)
model.learn(total_timesteps=50000)
# Store the trained Model and environment stats (which are needed as we are standardizing the observations and reward using VecNormalize())
model.save(RL_PATH + 'model')
env.save(RL_PATH + 'env.pkl')

# Evaluation

In [ ]:
# Plot the training process
training_log = pd.read_csv(RL_PATH + 'monitor.csv', skiprows=1)
training_log['r'].plot()

In [ ]:
env.training = False

actions, observations, electricity_consumption, excess_energy, rewards = ([], [], [], [], [])
done = False
obs = env.reset()
while not done:
        action = model.predict(obs, deterministic=True)
        obs, r, done, info = env.step([action[0][0]])

        actions.append(action[0][0][0])
        original_reward = env.get_original_reward()[0]
        original_obs = env.get_original_obs()[0]
        observations.append(original_obs)
        electricity_consumption.append(info[0]['electricity_consumption'])
        excess_energy.append(info[0]['electricity_price'])
        rewards.append(r)
        
trajectory = pd.DataFrame({
        'action': actions,
        'observations': observations,
        'electricity_consumption': electricity_consumption,
        'electricity_price': excess_energy,
        'reward': rewards
    })        

In [ ]:
plot_data = trajectory[0:200]
observation_df = plot_data['observations'].apply(pd.Series)

plt.rcParams["figure.figsize"] = (16,10)

fig, ax = plt.subplots()
ax.plot(observation_df[1], label = 'electric load')
ax.plot(observation_df[5], label = 'solar generation')
ax.plot(plot_data['electricity_price'], label = 'electricity_price')

ax1 = ax.twinx()
ax1.plot(plot_data['action'], label = 'action', color = 'black')
fig.legend(bbox_to_anchor=[0.5, 0.95], loc = 'center', ncol=5, prop={'size': 16})

# Compare to Baseline

In [ ]:
env.training = False

cost = []
done = False
obs = env.reset()
while not done:
        action = model.predict(obs, deterministic=True)
        obs, r, done, info = env.step([action[0][0]])
        cost.append(info[0]['electricity_consumption'] * info[0]['electricity_price'])

cost = sum(cost)

In [ ]:
env.training = False

baseline_cost = []
done = False
obs = env.reset()
while not done:
        # Always taking noop as action. This is the electricity demand if there would be no battery
        action = [0]
        obs, r, done, info = env.step(action)
        baseline_cost.append(info[0]['electricity_consumption'] * info[0]['electricity_price'])

baseline_cost = sum(baseline_cost)

In [ ]:
# how much energy did we save by utilizing the battery?
cost / baseline_cost